### Project  Movie Prophet
#### A project to create a predictive model to forecast a movie's performance on the Box Office
#### Team Members - Nelson Dsouza, Maria George, Saurabh Seth

##### movies_final.csv contains the final csv file for modelling

In [40]:
# GET THE LATEST CSV HERE
import pandas as pd
movie_df = pd.read_csv("movies_final.csv")
print(movie_df.shape)
movie_df.head()

#movie_df[movie_df['title'] == '10']

(6754, 14)


,key_bom,key_wiki,key_omdb,title,year,budget,gross,revenue,release_date,release_month,release_quarter,release_day_of_the_year,release_week_of_the_year,holiday_season
0,b27,w123,NaN,"""""""10,000 bc """"""",2008,105000000,269800000,1.569524,2008-03-07 00:00:00,3,1,67,9,0
1,b38,w299,o10558,102 dalmatians,2000,85000000,183600000,1.160000,2000-11-22 00:00:00,11,4,327,47,1
2,b48,w147,NaN,12 years a slave,2013,17100000,187700000,9.976608,2013-10-18 00:00:00,10,4,291,41,1
3,b50,w302,o7878,127 hours,yes,18000000,60700000,2.372222,2010-11-05 00:00:00,11,4,309,44,1
4,b55,w160,NaN,13 hours: the secret soldiers of benghazi,2016,50000000,70000000,0.400000,2016-01-15 00:00:00,1,1,15,2,1


In [23]:
movie_df[movie_df['revenue'].isnull()]

,key_bom,key_wiki,key_omdb,title,year,budget,gross,revenue


In [ ]:
movie_df[movie_df['revenue']==0]

### Common

In [1]:
# Load libraries and instatiate DB connection

%matplotlib inline
import pandas as pd
import pymysql
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.max_seq_items = 2000

# Connect to the database
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='mprophet',
                             db='movies')

In [2]:
# Pull table from database

cur = connection.cursor()

cur.execute("select * FROM bom")
bom_sql = []
for row in cur: bom_sql.append(list(row))

cur.execute("select * FROM wiki")
wiki_sql = []
for row in cur: wiki_sql.append(list(row))
    
cur.execute("select * FROM comb_index")
comb_sql = []
for row in cur: comb_sql.append(list(row))
    
cur.execute("select * FROM movies_final")
movie_sql = []
for row in cur: movie_sql.append(list(row))
    
cur.close()
connection.close()

In [3]:
# Store bom table information in DataFrame

bom_columns =         ['bom_title', 'bom_link', 'key_bom', 'bom_date', 'bom_year', 'bom_name', 'bom_budget',
                       'bom_domestic_gross', 'bom_lifetime_gross', 'bom_release_date', 'bom_runtime', 'bom_distributor',
                       'bom_genre', 'bom_rating', 'bom_awards', 'bom_actor', 'bom_assoc_producer', 'bom_cinematographer',
                       'bom_composer', 'bom_director', 'bom_exec_producer', 'bom_line_producer', 'bom_players', 'bom_producer',
                       'bom_writer', 'bom_close_date', 'bom_domestic_summary', 'bom_in_release', 'bom_limited_opening_weekend',
                       'bom_opening_weekend', 'bom_wide_opening_weekend', 'bom_widest_release', 'bom_domestic_revenue',
                       'bom_foreign_revenue', 'bom_worldwide_revenue', 'bom_total_lifetime_gross']

bom = pd.DataFrame(bom_sql, columns=bom_columns)
#bom.to_csv('bom.csv')
print('Total movies from Box Office Mojo:', len(bom))
bom.head(1)

Total movies from Box Office Mojo: 16826


,bom_title,bom_link,key_bom,bom_date,bom_year,bom_name,bom_budget,bom_domestic_gross,bom_lifetime_gross,bom_release_date,bom_runtime,bom_distributor,bom_genre,bom_rating,bom_awards,bom_actor,bom_assoc_producer,bom_cinematographer,bom_composer,bom_director,bom_exec_producer,bom_line_producer,bom_players,bom_producer,bom_writer,bom_close_date,bom_domestic_summary,bom_in_release,bom_limited_opening_weekend,bom_opening_weekend,bom_wide_opening_weekend,bom_widest_release,bom_domestic_revenue,bom_foreign_revenue,bom_worldwide_revenue,bom_total_lifetime_gross
0,#horror,http://www.boxofficemojo.com/movies/?id=horrorifc.htm,b0,20-Nov-15,2015\r,#Horror,N/A,,,20-Nov-15,1 hrs. 30 min.,IFC,Horror,Unknown,No Academy Awards Information\r,"""['Balthazar Getty', 'Timothy Hutton', 'Natasha Lyonne', 'Taryn Manning', 'Chloe Sevigny']""",,,,,,,,,\r,,No Domestic Summary Information,,,,,\r,,,,No Total Lifetime Gross Information\r


In [4]:
wiki_columns = ['wiki_budget',
                "wiki_cinematography","wiki_director","wiki_distributor", 
                "wiki_editing", "wiki_gross" , "wiki_music", 
                "wiki_pageid", "wiki_producer", "wiki_released", 
                "wiki_runtime", "wiki_screenplay", "wiki_starring", 
                "wiki_studio","wiki_title", "key_wiki", "year_wiki",
                "year_wiki_format"];

wiki = pd.DataFrame(wiki_sql, columns=wiki_columns)
#bom.to_csv('bom.csv')
print('Total movies from Wikipedia:', len(wiki))

wiki['key_wiki'] = [str(v).lstrip() for v in wiki['key_wiki']]
wiki['key_wiki']= [str(v).rstrip() for v in wiki['key_wiki']]

wiki.head(1)

Total movies from Wikipedia: 40981


,wiki_budget,wiki_cinematography,wiki_director,wiki_distributor,wiki_editing,wiki_gross,wiki_music,wiki_pageid,wiki_producer,wiki_released,wiki_runtime,wiki_screenplay,wiki_starring,wiki_studio,wiki_title,key_wiki,year_wiki,year_wiki_format
0,,"""[Gilbert Salas, Heather Rae]""",[Heather Rae],[],"""[Gregory Bayne, Heather Rae]""",,[John Trudell & Bad Dog],5975700,"""[Heather Rae, Elyse Katz]""","""[2005, 01, 20, Sundance Film Festival, 2006, 02, 24, United States]""",[80 minutes],,[],,trudell,w39443,None,None


In [5]:
comb_columns = ["key_bom","key_wiki","key_omdb","bom_title","wiki_title","omdb_title","year_bom",
               "year_bom_format","year_wiki", "year_wiki_format", "year_omdb", "year_omdb_format","score"]
comb_index = pd.DataFrame(comb_sql, columns=comb_columns)
#bom.to_csv('bom.csv')
print('Total movies in combined file:', len(comb_index))
comb_index.head()

Total movies in combined file: 24665


,key_bom,key_wiki,key_omdb,bom_title,wiki_title,omdb_title,year_bom,year_bom_format,year_wiki,year_wiki_format,year_omdb,year_omdb_format,score
0,b15584,w39443,o18495,trudell,trudell,trudell,3-Feb-06,2006,2005,2005,20-Jan-05,2005,1\r
1,b14216,w28904,o12944,the party at kitty and stud's,the party at kitty and stud's,the party at kitty and stud's,6-Jul-76,1976,1970,1970,10-Feb-70,1970,1\r
2,,w4957,o17832,,borrowed wives,borrowed wives,,,20-Aug,20-Aug,20-Aug-30,1930,1\r
3,,w40302,o15689,,the vagabond trail,the vagabond trail,,,9-Mar,9-Mar,9-Mar-24,1924,1\r
4,,w11443,o9690,,eddie's million dollar cook-off,eddie's million dollar cook-off,,,2003,2003,18-Jul-03,2003,1\r


In [6]:
movie_columns = ["key_bom","key_wiki","key_omdb","bom_title","wiki_title","omdb_title",
                                  "year_bom_format","year_wiki_format","year_omdb_format",
                                "wiki_budget","wiki_gross","bom_budget","bom_domestic_gross"]
movie_df = pd.DataFrame(movie_sql, columns=movie_columns)
#bom.to_csv('bom.csv')
print('Total movies in combined file:', len(movie_df))
movie_df.head()

Total movies in combined file: 7995


,key_bom,key_wiki,key_omdb,bom_title,wiki_title,omdb_title,year_bom_format,year_wiki_format,year_omdb_format,wiki_budget,wiki_gross,bom_budget,bom_domestic_gross
0,b10,,,(500) days of summer,,,,,,,,$7.5 million,32391374
1,b24,,,10 things i hate about you,,,,,,,,$30 million,38178166
2,b27,w123,,"""""""10,000 b.c.""""""","""""""10,000 bc """"""",,2008,2008,,[$105 million],[$269.8 million],$105 million,94784201
3,b38,w299,o10558,102 dalmatians,102 dalmatians,102 dalmatians,2000,2000,2000,[$85 million],[$183.6 million],$85 million,66957026
4,b39,,,10th &amp; wolf,,,,,,,,$8 million,54702


### Nelson

In [ ]:
# Retrieving the base table and only taking entries which have key for bom
movies_f = movie_df[~movie_df['key_bom'].isnull()]
#movies_f = movies_f[~movies_f['revenue'].isnull()]
print('Movies with revenue information for bom', len(movies_f))

# Cleaning actor names and storing in list
bom_ftr = bom.copy()
print('Movies with no player information:', len(bom_ftr[bom_ftr['bom_players'] == 'No Player Information']))
bom_ftr['actor_clean'] = bom_ftr['bom_actor'].str.replace('[','')
bom_ftr['actor_clean'] = bom_ftr['actor_clean'].str.replace(']','')
bom_ftr['actor_clean'] = bom_ftr['actor_clean'].str.replace('"','')
bom_ftr['actor_clean'] = bom_ftr['actor_clean'].str.replace("'",'')
bom_ftr['actor_clean'] = bom_ftr['actor_clean'].str.replace(", ",',')
bom_ftr['actor_clean'] = bom_ftr['actor_clean'].str.replace("*",',')
bom_ftr['actor_clean'] = bom_ftr['actor_clean'].apply(lambda x: x.split(','))

movies_f = pd.merge(bom_ftr, movies_f,on = "key_bom")
movies_f.head(2)

In [ ]:
'''
# Calclulate the revenue ratio for each movie which has data for budget and domestic total gross 

print('Movies with no budget information:', len(bom_ftr[bom_ftr['bom_budget'] == 'N/A']))
print('Movies with no gross total information:', len(bom_ftr[bom_ftr['bom_domestic_gross'] == '']))

bom_ftr_rev = bom_ftr[(bom_ftr['bom_budget'] != 'N/A') & (bom_ftr['bom_domestic_gross'] != '')].copy()
bom_ftr_rev.reset_index(inplace=True)
bom_ftr_rev['bom_budget'] = bom_ftr_rev['bom_budget'].str.replace('$','')
bom_ftr_rev['bom_budget'] = bom_ftr_rev['bom_budget'].str.replace('"','')
bom_ftr_rev['bom_budget'] = bom_ftr_rev['bom_budget'].str.replace(',','')

bom_ftr_rev.loc[bom_ftr_rev.bom_budget.str.contains('million'),'bom_budget'] = bom_ftr_rev.loc[
    bom_ftr_rev.bom_budget.str.contains('million'), 'bom_budget'].apply(lambda x: float(x.replace('million', ''))*1000000)

bom_ftr_rev['bom_domestic_gross'] = bom_ftr_rev['bom_domestic_gross'].astype(float)
bom_ftr_rev['bom_budget'] = bom_ftr_rev['bom_budget'].astype(float)
bom_ftr_rev['bom_revenue'] = (bom_ftr_rev['bom_domestic_gross'] - bom_ftr_rev['bom_budget']) / bom_ftr_rev['bom_budget']

print('\nMovies with revenue information:', len(bom_ftr_rev))
bom_ftr_rev.head(1)
'''

In [ ]:
# Getting unique actors
actor_unq = list(movies_f['actor_clean'])
actor_unq = list(set(sum(actor_unq, [])))
actor_unq = [e for e in actor_unq if e not in ('', ' ')]
actor_unq = dict((ky,0) for ky in actor_unq)
print('Unique actors in bom: ', len(actor_unq))

# For each actor, assigning score based on revenue of their movie
mov_actors = list(movies_f['actor_clean'])
mov_actors = [[actor for actor in mov if actor not in ('', ' ')] for mov in mov_actors]

ind = 0
for actors in mov_actors:
    for actor in actors:
        actor_unq[actor] += float(movies_f.loc[ind, ['revenue']])
    ind+=1

In [ ]:
actor_unq['Meryl Streep']

In [ ]:
# Pull the actor feature into dataframe and calculate stats

f_actor = pd.Series(actor_unq, name='score').to_frame()
f_actor = f_actor.sort_values(by='score', ascending=False)
f_actor.reset_index(inplace=True)
f_actor.columns=['Actors', 'score']
print(f_actor.head(5))
f_actor.describe().transpose()

In [ ]:
# Analyzing distribution

plt.figure(1, figsize = (15,6))
plt.hist(list(f_actor[f_actor['score'] != 0]['score']), range=[0,30], bins=(30), facecolor='purple')
plt.title("Distribution of Actor Scores", fontsize = 22)
plt.ylabel("Score", fontsize = 15)
plt.xlabel("Number of Actors", fontsize=15)

ax = plt.gca()
ax.spines['right'].set_color('none')
ax.spines['top'].set_color('none')
ax.yaxis.set_ticks_position('none')
ax.xaxis.set_ticks_position('none')

plt.savefig('Actor_Distribution.png')

In [ ]:
# For each movie, assigning actor score

movies_f["bom_actor_score"] = 0.0
ind = 0
for actors in mov_actors:
    agg_actor_score = 0
    for actor in actors:
        agg_actor_score += actor_unq[actor]
    movies_f.loc[ind, ['bom_actor_score']] = agg_actor_score
    ind+=1
    
print(movies_f.shape)
movies_f.head(1)

In [ ]:
print(len(movie_df[~movie_df['key_bom'].isnull()]))
print(len(set(movie_df[~movie_df['key_bom'].isnull()]['key_bom'])))

print(len(movie_df[~bom_ftr['key_bom'].isnull()]))
print(len(set(movie_df[~bom_ftr['key_bom'].isnull()]['key_bom'])))

In [ ]:
movie_df = pd.read_csv("movies_final.csv")
print(movie_df.shape)

movies_score = movies_f[['key_bom', 'bom_actor_score']]
print(movies_score.shape)

movie_df = pd.merge(movie_df, movies_score,on = "key_bom", how='left')
movie_df.shape
movie_df.head(2)

### Maria

###### Creating the combined data (to be loaded into the table movies_final)

In [ ]:
# Removing duplicate rows from wiki
wiki = wiki.fillna("")
print(wiki['key_wiki'].unique().shape)
print(wiki.shape)
wiki = wiki.drop_duplicates(['key_wiki'],keep ='last')
print(wiki.shape)

In [ ]:
# Chexking duplicates in comb_df
#comb_index = comb_index.fillna("*")
print(comb_index.shape)

print('bom unique keys',comb_index['key_bom'].unique().shape)
a = comb_index[comb_index['key_bom']!=""]
print('bom not null',a.shape)

# print the rows with where bom ky is duplicated
k = a[a.duplicated(['key_bom'])]['key_bom'].tolist()
#comb_index[comb_index['key_bom'].isin(k)]

# deleting duplicate rows
comb_index = comb_index.drop(comb_index.index[[7626,16978,22685,23066,23068,23155,23975,24029,24036,24054,24093,
                                              24103,24120,24123,24174,24257,24268,24269]])

comb_index.reset_index(inplace = True,drop =True)
print('Shape, combined_index after dropping bom duplicates',comb_index.shape)

In [ ]:
print('omdb unique keys',comb_index['key_omdb'].unique().shape)
a = comb_index[comb_index['key_omdb']!=""]
print('omdb not null',a.shape)

# print the rows with where bom ky is duplicated
k = a[a.duplicated(['key_omdb'])]['key_omdb'].tolist()
comb_index[comb_index['key_omdb'].isin(k)]

# deleting duplicate rows
comb_index = comb_index.drop(comb_index.index[[15696,21655]])

comb_index.reset_index(inplace = True,drop =True)
print('Shape, combined_index after dropping omdb duplicates',comb_index.shape)


In [ ]:
print('wiki unique keys',comb_index['key_wiki'].unique().shape)
a = comb_index[comb_index['key_wiki']!=""]
print('wiki not null',a.shape)

# print the rows with where bom ky is duplicated
k = a[a.duplicated(['key_wiki'])]['key_wiki'].tolist()
comb_index[comb_index['key_wiki'].isin(k)]

# deleting duplicate rows
comb_index = comb_index.drop(comb_index.index[[1895,22386,22420,23060,23091,23854,23901,23944,24000,24006,
                                              24016,24022,24025,24066,24090,24094,24171,24264,24299,24383,24402,
                                              24454,24619,24643,24289,24401]])

comb_index.reset_index(inplace = True,drop =True)
print('Shape, combined_index after dropping wiki duplicates',comb_index.shape)


In [ ]:
print(comb_index['key_bom'].unique().shape)
print(comb_index['key_wiki'].unique().shape)
print(comb_index['key_omdb'].unique().shape)

print(comb_index.groupby(['key_omdb','key_bom','key_wiki']).count().shape)

comb_index = comb_index.drop_duplicates(['key_omdb','key_bom','key_wiki'],keep='last')
comb_index.shape

In [ ]:
# Filtering wiki for data for which wiki_budget and wiki_gross exists
print("no:of rows in wiki where budget and gross revenue is not null:")
print(wiki[(wiki.wiki_budget!="") & (wiki.wiki_gross!="")].shape)

print("no:of rows in bom where budget and gross revenue is not null:")
bom['bom_budget'] = [str(v).lstrip() for v in bom['bom_budget']]
bom['bom_budget']= [str(v).rstrip() for v in bom['bom_budget']]
f = bom.bom_domestic_gross
print(bom[(bom.bom_budget!="") & (bom.bom_budget!="N/A") & (f!="")].shape)

# Filtering the rows and intersted columns
wiki_f = wiki[(wiki.wiki_budget!="") & (wiki.wiki_gross!="")]
wiki_f = wiki_f[["key_wiki","wiki_title","wiki_budget","wiki_gross"]]
print(wiki_f.shape)
bom_f = bom[(bom.bom_budget!="") & (bom.bom_budget!="N/A") & (f!="")]
bom_f = bom_f[["key_bom","bom_title","bom_budget","bom_domestic_gross"]]
print(bom_f.shape)

In [ ]:
# Merging to the combined index
comb_index_f = comb_index[["key_bom","key_wiki","key_omdb","omdb_title","year_bom_format",
                           "year_wiki_format","year_omdb_format"]]

wiki_f_comb = pd.merge(wiki_f,comb_index_f,on = "key_wiki",how="left")
print(wiki_f_comb.shape)
wiki_f_comb.head()

In [ ]:
wiki_bom_f_comb = pd.merge(bom_f,wiki_f_comb,on = "key_bom",how="outer")
print(wiki_bom_f_comb.shape)
wiki_bom_f_comb = wiki_bom_f_comb.fillna("")
wiki_bom_f_comb.head()

In [ ]:
# Checking one last time 

print(wiki_bom_f_comb['key_bom'].unique().shape)
print(wiki_bom_f_comb['key_wiki'].unique().shape)
print(wiki_bom_f_comb['key_omdb'].unique().shape)

print(wiki_bom_f_comb.groupby(['key_omdb','key_bom','key_wiki']).count().shape)


In [ ]:
wiki_bom_f_comb = wiki_bom_f_comb[["key_bom","key_wiki","key_omdb","bom_title","wiki_title","omdb_title",
                                  "year_bom_format","year_wiki_format","year_omdb_format",
                                  "wiki_budget","wiki_gross","bom_budget","bom_domestic_gross"]]
wiki_bom_f_comb.to_csv("CombinedFilev1.csv",index = None)
wiki_bom_f_comb.to_csv("combinedfilev1.txt",sep="\t",index = None)

In [ ]:
##### BELOW CELL DEPRECATED

In [ ]:
# Filtering wiki for data for which wiki_budget and wiki_gross exists
print("no:of rows in wiki where budget and gross revenue is not null:")
print(wiki[(wiki.wiki_budget!="") & (wiki.wiki_gross!="")].shape)

print("no:of rows in bom where budget and gross revenue is not null:")
bom['bom_budget'] = [str(v).lstrip() for v in bom['bom_budget']]
bom['bom_budget']= [str(v).rstrip() for v in bom['bom_budget']]
f = bom.bom_domestic_gross
print(bom[(bom.bom_budget!="") & (bom.bom_budget!="N/A") & (f!="")].shape)

# Filtering the rows and intersted columns
wiki_f = wiki[(wiki.wiki_budget!="") & (wiki.wiki_gross!="")]
wiki_f = wiki_f[["key_wiki","wiki_title","wiki_budget","wiki_gross"]]
bom_f = bom[(bom.bom_budget!="") & (bom.bom_budget!="N/A") & (f!="")]
bom_f = bom_f[["key_bom","bom_title","bom_budget","bom_domestic_gross"]]


# Filtering intersted columsn from 
comb_index_f = comb_index[["key_bom","key_wiki","key_omdb","omdb_title","year_bom_format",
                           "year_wiki_format","year_omdb_format"]]

# Merging the combined index table
wiki_f_comb = pd.merge(wiki_f,comb_index_f,on = "key_wiki")
print(wiki_f_comb.shape)
bom_f_comb = pd.merge(bom_f,comb_index_f,on = "key_bom")
print(bom_f_comb.shape)

wiki_bom_f_comb = pd.merge(wiki_f_comb,bom_f_comb,on = ["key_wiki","key_bom"])
wiki_bom_f_comb = wiki_bom_f_comb.rename(columns = {'key_omdb_x':'key_omdb'})
wiki_bom_f_comb = wiki_bom_f_comb.drop('key_omdb_y',axis=1)
wiki_bom_f_comb = wiki_bom_f_comb.rename(columns = {'omdb_title_x':'omdb_title'})
wiki_bom_f_comb = wiki_bom_f_comb.drop('omdb_title_y',axis=1)
wiki_bom_f_comb = wiki_bom_f_comb.rename(columns = {'year_omdb_format_x':'year_omdb_format'})
wiki_bom_f_comb = wiki_bom_f_comb.drop('year_omdb_format_y',axis=1)
wiki_bom_f_comb = wiki_bom_f_comb.rename(columns = {'year_bom_format_x':'year_bom_format'})
wiki_bom_f_comb = wiki_bom_f_comb.drop('year_bom_format_y',axis=1)
wiki_bom_f_comb = wiki_bom_f_comb.rename(columns = {'year_wiki_format_x':'year_wiki_format'})
wiki_bom_f_comb = wiki_bom_f_comb.drop('year_wiki_format_y',axis=1)
#print (wiki_bom_f_comb.columns)
print("Combined v1:", wiki_bom_f_comb.shape)

# count of unique wiki,bom key
#wiki_bom_f_comb.groupby(['key_wiki', 'key_bom']).size().shape
#wiki_bom_f_comb.head()

# Appending the rest
wiki_f_comb_rem = wiki_f_comb[~wiki_f_comb['key_wiki'].isin(wiki_bom_f_comb['key_wiki'])]
print(wiki_f_comb_rem.shape)
bom_f_comb_rem = bom_f_comb[~bom_f_comb['key_bom'].isin(wiki_bom_f_comb['key_bom'])]
print(bom_f_comb_rem.shape)

wiki_bom_f_comb = wiki_bom_f_comb.append(wiki_f_comb_rem)
wiki_bom_f_comb = wiki_bom_f_comb.append(bom_f_comb_rem)
print("Combined v2:", wiki_bom_f_comb.shape)

wiki_f_rem = wiki_f[~wiki_f['key_wiki'].isin(wiki_bom_f_comb['key_wiki'])]
print(wiki_f_rem.shape)
bom_f_rem = bom_f[~bom_f['key_bom'].isin(wiki_bom_f_comb['key_bom'])]
print(bom_f_rem.shape)

wiki_bom_f_comb = wiki_bom_f_comb.append(wiki_f_rem)
wiki_bom_f_comb = wiki_bom_f_comb.append(bom_f_rem)
print("Combined v3:", wiki_bom_f_comb.shape)

# Rearranging the columsn
#print (wiki_bom_f_comb.columns)
wiki_bom_f_comb = wiki_bom_f_comb[["key_bom","key_wiki","key_omdb","bom_title","wiki_title","omdb_title",
                                  "year_bom_format","year_wiki_format","year_omdb_format",
                                  "wiki_budget","wiki_gross","bom_budget","bom_domestic_gross"]]

wiki_bom_f_comb.to_csv("CombinedFile.csv",index = None)
wiki_bom_f_comb.to_csv("combinedfile.txt",sep="\t",index = None)
wiki_bom_f_comb.head()

###### Cleaning the revenue, budget column

In [7]:
# Cleaning bom budget and gross columns

movie_df.loc[movie_df.bom_budget == "",'bom_budget']= movie_df.loc[movie_df.bom_budget=="",'bom_budget'].apply(lambda x: float(x.replace('', '0.0')))

movie_df.loc[movie_df.bom_domestic_gross == "",'bom_domestic_gross']= movie_df.loc[movie_df.bom_domestic_gross=="",'bom_domestic_gross'].apply(lambda x: float(x.replace('', '0.0')))

movie_df = movie_df.fillna(0)

#movie_df.reset_index(inplace=True)
movie_df['bom_budget'] = movie_df['bom_budget'].str.replace('$','')
movie_df['bom_budget'] = movie_df['bom_budget'].str.replace('"','')
movie_df['bom_budget'] = movie_df['bom_budget'].str.replace(',','')

movie_df.loc[movie_df.bom_budget.str.contains('million',na=False),'bom_budget'] = movie_df.loc[
    movie_df.bom_budget.str.contains('million',na=False), 'bom_budget'].apply(lambda x: float(x.replace('million', ''))*1000000)

movie_df['bom_domestic_gross'] = movie_df['bom_domestic_gross'].astype(float)
movie_df['bom_budget'] = movie_df['bom_budget'].astype(float)
movie_df.head()

,key_bom,key_wiki,key_omdb,bom_title,wiki_title,omdb_title,year_bom_format,year_wiki_format,year_omdb_format,wiki_budget,wiki_gross,bom_budget,bom_domestic_gross
0,b10,,,(500) days of summer,,,,,,,,7500000,32391374
1,b24,,,10 things i hate about you,,,,,,,,30000000,38178166
2,b27,w123,,"""""""10,000 b.c.""""""","""""""10,000 bc """"""",,2008,2008,,[$105 million],[$269.8 million],105000000,94784201
3,b38,w299,o10558,102 dalmatians,102 dalmatians,102 dalmatians,2000,2000,2000,[$85 million],[$183.6 million],85000000,66957026
4,b39,,,10th &amp; wolf,,,,,,,,8000000,54702


In [8]:
# Cleaning wikipedia budget and gross column

import warnings
warnings.filterwarnings('ignore')

t = movie_df
t['wiki_budget'] = t['wiki_budget'].str.replace("million","#")
t['wiki_gross'] = t['wiki_gross'].str.replace("million","#")

def make_hidden(instr):
    return ''.join(c if not c.isalpha() else '' for c in instr)
import re
def make_hidden_1(instr):
    return ''.join(re.sub('[^#.0-9]+', '', instr))

t['wiki_budget'] = t['wiki_budget'].apply(make_hidden)
t['wiki_budget'] = t['wiki_budget'].apply(make_hidden_1)
t['wiki_gross'] = t['wiki_gross'].apply(make_hidden)
t['wiki_gross'] = t['wiki_gross'].apply(make_hidden_1)

def counter(instr):
    return str(instr).count('.')

for k,v in t.wiki_gross.iteritems():
    if(counter(v)>=2):
        t = t.drop(k)
        
for k,v in t.wiki_budget.iteritems():
    if(counter(v)>=2):
        t = t.drop(k)

def make_hidden_2(instr):
    if instr!="":
        if instr[-1]==".":
            return instr[:-1]
        else:
            return instr
    
t['wiki_budget'] = t['wiki_budget'].apply(make_hidden_2)
t['wiki_gross'] = t['wiki_gross'].apply(make_hidden_2)
        
t.loc[t.wiki_budget.str.contains('#',na=False),'wiki_budget'] = t.loc[
    t.wiki_budget.str.contains('#',na=False), 'wiki_budget'].apply(lambda x: float(x.replace('#', ''))*1000000)

t.loc[t.wiki_gross.str.contains('#',na=False),'wiki_gross'] = t.loc[
    t.wiki_gross.str.contains('#',na=False), 'wiki_gross'].apply(lambda x: float(x.replace('#', ''))*1000000)

t.loc[t.wiki_budget == "",'wiki_budget']= t.loc[t.wiki_budget=="",'wiki_budget'].apply(lambda x: float(x.replace('', '0.0')))

t.loc[t.wiki_gross == "",'wiki_gross']= t.loc[t.wiki_gross=="",'wiki_gross'].apply(lambda x: float(x.replace('', '0.0')))

#print(t.loc[t.wiki_budget.str.contains('.',na=False),'wiki_budget'])

t['wiki_budget'] = t['wiki_budget'].astype(float)
t['wiki_gross'] = t['wiki_gross'].astype(float)

movie_df = t
print(movie_df.shape)
movie_df.head()

(7934, 13)


,key_bom,key_wiki,key_omdb,bom_title,wiki_title,omdb_title,year_bom_format,year_wiki_format,year_omdb_format,wiki_budget,wiki_gross,bom_budget,bom_domestic_gross
0,b10,,,(500) days of summer,,,,,,NaN,NaN,7500000,32391374
1,b24,,,10 things i hate about you,,,,,,NaN,NaN,30000000,38178166
2,b27,w123,,"""""""10,000 b.c.""""""","""""""10,000 bc """"""",,2008,2008,,105000000,269800000,105000000,94784201
3,b38,w299,o10558,102 dalmatians,102 dalmatians,102 dalmatians,2000,2000,2000,85000000,183600000,85000000,66957026
4,b39,,,10th &amp; wolf,,,,,,NaN,NaN,8000000,54702


In [9]:
movie_df.to_csv("movies_final.csv",index=None)

##### Pick only one title,year

In [11]:
print(movie_df.shape)
movie_df = movie_df.fillna("")
movie_df.head()

(7934, 13)


,key_bom,key_wiki,key_omdb,bom_title,wiki_title,omdb_title,year_bom_format,year_wiki_format,year_omdb_format,wiki_budget,wiki_gross,bom_budget,bom_domestic_gross
0,b10,,,(500) days of summer,,,,,,,,7.5e+06,32391374
1,b24,,,10 things i hate about you,,,,,,,,3e+07,38178166
2,b27,w123,,"""""""10,000 b.c.""""""","""""""10,000 bc """"""",,2008,2008,,1.05e+08,2.698e+08,1.05e+08,94784201
3,b38,w299,o10558,102 dalmatians,102 dalmatians,102 dalmatians,2000,2000,2000,8.5e+07,1.836e+08,8.5e+07,66957026
4,b39,,,10th &amp; wolf,,,,,,,,8e+06,54702


In [12]:
for r in range(0,movie_df.shape[0]):
    if(movie_df.loc[r,'key_wiki']!=""):
        movie_df.loc[r,'title']=movie_df.loc[r,'wiki_title']
    elif(movie_df.loc[r,'key_bom']!=""):
        movie_df.loc[r,'title']=movie_df.loc[r,'bom_title']
    else:
        movie_df.loc[r,'title']=movie_df.loc[r,'omdb_title']

for r in range(0,movie_df.shape[0]):
    if(movie_df.loc[r,'year_wiki_format']!=""):
        movie_df.loc[r,'year']=movie_df.loc[r,'year_wiki_format']
    elif(movie_df.loc[r,'year_bom_format']!=""):
        movie_df.loc[r,'year']=movie_df.loc[r,'year_bom_format']
    else:
        movie_df.loc[r,'year']=movie_df.loc[r,'year_omdb_format']

print(movie_df.shape)

movie_df = movie_df[movie_df['title']!=""]
movie_df = movie_df[movie_df['key_wiki'].str.startswith('w')]
movie_df.reset_index(inplace=True,drop=True)
print(movie_df.shape)

# Removing the redunant columns
movie_df = movie_df.drop('bom_title',axis = 1)
movie_df = movie_df.drop('wiki_title',axis = 1)
movie_df = movie_df.drop('omdb_title',axis = 1)

movie_df = movie_df.drop('year_bom_format',axis = 1)
movie_df = movie_df.drop('year_wiki_format',axis = 1)
movie_df = movie_df.drop('year_omdb_format',axis = 1)

print(movie_df.shape)


(7934, 15)
(6784, 15)
(6784, 9)


In [13]:
movie_df.to_csv("movies_final.csv",index=None)

##### Pick only one budget and gross

In [15]:
t = movie_df
t = t.fillna("")

for r in range(0,t.shape[0]):
    if(t.loc[r,'key_bom']!=""):
        if(t.loc[r,'key_wiki']==""):
            t.loc[r,'budget']=t.loc[r,'bom_budget']
            t.loc[r,'gross']=t.loc[r,'bom_domestic_gross']
        else:
            t.loc[r,'budget']=t.loc[r,'wiki_budget']
            t.loc[r,'gross']=t.loc[r,'wiki_gross']
    else:
        t.loc[r,'budget']=t.loc[r,'wiki_budget']
        t.loc[r,'gross']=t.loc[r,'wiki_gross']
        
t.head()

,key_bom,key_wiki,key_omdb,wiki_budget,wiki_gross,bom_budget,bom_domestic_gross,title,year,budget,gross
0,b27,w123,,1.05e+08,2.698e+08,1.05e+08,94784201,"""""""10,000 bc """"""",2008,1.05e+08,2.698e+08
1,b38,w299,o10558,8.5e+07,1.836e+08,8.5e+07,66957026,102 dalmatians,2000,8.5e+07,1.836e+08
2,b48,w147,,1.71e+07,1.877e+08,2e+07,56671993,12 years a slave,2013,1.71e+07,1.877e+08
3,b50,w302,o7878,1.8e+07,6.07e+07,1.8e+07,18335230,127 hours,yes,1.8e+07,6.07e+07
4,b55,w160,,5e+07,7e+07,5e+07,52853219,13 hours: the secret soldiers of benghazi,2016,5e+07,7e+07


In [16]:
movie_df = t
# Removing the redunant columns
movie_df = movie_df.drop('bom_budget',axis = 1)
movie_df = movie_df.drop('wiki_budget',axis = 1)
movie_df = movie_df.drop('bom_domestic_gross',axis = 1)
movie_df = movie_df.drop('wiki_gross',axis = 1)

print(movie_df.shape)
movie_df.to_csv("movies_final.csv",index=None)

(6784, 7)


###### Cleaning gross, budget column

In [18]:
movie_df = movie_df[~movie_df['gross'].isnull()]
movie_df = movie_df[~movie_df['budget'].isnull()]
print(movie_df.shape)
movie_df.to_csv("movies_final.csv",index=None)

(6726, 7)


###### Creating the response variable: revenue

In [20]:
movie_df['revenue'] = (movie_df['gross'] - movie_df['budget']) / movie_df['budget']
movie_df.head()

,key_bom,key_wiki,key_omdb,title,year,budget,gross,revenue
0,b27,w123,NaN,"""""""10,000 bc """"""",2008,105000000,269800000,1.569524
1,b38,w299,o10558,102 dalmatians,2000,85000000,183600000,1.160000
2,b48,w147,NaN,12 years a slave,2013,17100000,187700000,9.976608
3,b50,w302,o7878,127 hours,yes,18000000,60700000,2.372222
4,b55,w160,NaN,13 hours: the secret soldiers of benghazi,2016,50000000,70000000,0.400000


In [21]:
movie_df.to_csv("movies_final.csv",index=None)

###### Features related to Release date

###### 1. Creating a single column release_date
###### 2. Converts to datetime
###### 3. Features: month, quarter, week of the year, day of the year

In [24]:
# comb_index contains unformatted release date
print(comb_index.shape)
comb_index.head()

(24665, 13)


,key_bom,key_wiki,key_omdb,bom_title,wiki_title,omdb_title,year_bom,year_bom_format,year_wiki,year_wiki_format,year_omdb,year_omdb_format,score
0,b15584,w39443,o18495,trudell,trudell,trudell,3-Feb-06,2006,2005,2005,20-Jan-05,2005,1\r
1,b14216,w28904,o12944,the party at kitty and stud's,the party at kitty and stud's,the party at kitty and stud's,6-Jul-76,1976,1970,1970,10-Feb-70,1970,1\r
2,,w4957,o17832,,borrowed wives,borrowed wives,,,20-Aug,20-Aug,20-Aug-30,1930,1\r
3,,w40302,o15689,,the vagabond trail,the vagabond trail,,,9-Mar,9-Mar,9-Mar-24,1924,1\r
4,,w11443,o9690,,eddie's million dollar cook-off,eddie's million dollar cook-off,,,2003,2003,18-Jul-03,2003,1\r


In [30]:
# merging comb_index with movie_df

comb_index_1 = comb_index[["key_bom","year_bom"]]
comb_index_2 = comb_index[["key_wiki","year_wiki"]]
comb_index_3 = comb_index[["key_omdb","year_omdb"]]

movie_df_1 = pd.merge(movie_df,comb_index_1, on = ["key_bom"],how="left")
print(movie_df_1.shape)
movie_df_2 = pd.merge(movie_df_1,comb_index_2, on = ["key_wiki"],how="left")
print(movie_df_2.shape)
movie_df_3 = pd.merge(movie_df_2,comb_index_3, on = ["key_omdb"],how="left")
print(movie_df_3.shape)

movie_df_3 = movie_df_3.fillna("")
movie_df_3.head()

(6732, 9)
(6753, 10)
(6754, 11)


,key_bom,key_wiki,key_omdb,title,year,budget,gross,revenue,year_bom,year_wiki,year_omdb
0,b27,w123,,"""""""10,000 bc """"""",2008,105000000,269800000,1.569524,7-Mar-08,[2008],
1,b38,w299,o10558,102 dalmatians,2000,85000000,183600000,1.160000,22-Nov-00,2000,22-Nov-00
2,b48,w147,,12 years a slave,2013,17100000,187700000,9.976608,18-Oct-13,"""[2013, 08, 30, Telluride Film Festival, 2013, 11, 08, United States, 2014, 01, 10, United Kingd...",
3,b50,w302,o7878,127 hours,yes,18000000,60700000,2.372222,5-Nov-10,yes,28-Jan-11
4,b55,w160,,13 hours: the secret soldiers of benghazi,2016,50000000,70000000,0.400000,15-Jan-16,"""[2016, 01, 12, AT&T Stadium, 2016, 01, 15, United States]""",


In [31]:
# Create a single column release_date
# Preference for bom, omdb and then wiki
# release_flag set to 0 if release date is fetched from bom or omdb. If fetched from wiki, then 1

for r in range(0,movie_df_3.shape[0]):
    if(movie_df_3.loc[r,'year_bom']!=""):
        movie_df_3.loc[r,'release_date']=movie_df_3.loc[r,'year_bom']
        #movie_df_3.loc[r,'release_flag']=0
    elif(movie_df_3.loc[r,'year_omdb']!=""):
        movie_df_3.loc[r,'release_date']=movie_df_3.loc[r,'year_omdb']
        #movie_df_3.loc[r,'release_flag']=0
    else:
        movie_df_3.loc[r,'release_date']=movie_df_3.loc[r,'year_wiki']
        #movie_df_3.loc[r,'release_flag']=1
        
movie_df_3 = movie_df_3.drop('year_bom',axis = 1)
movie_df_3 = movie_df_3.drop('year_omdb',axis = 1)
movie_df_3 = movie_df_3.drop('year_wiki',axis = 1)

movie_df_3.to_csv("checkreleasedate.csv",index=None)

#movie_df = movie_df_3[movie_df_3['release_date']!=""]
print(movie_df_3.shape)

movie_df_3.head()

(6754, 9)


,key_bom,key_wiki,key_omdb,title,year,budget,gross,revenue,release_date
0,b27,w123,,"""""""10,000 bc """"""",2008,105000000,269800000,1.569524,7-Mar-08
1,b38,w299,o10558,102 dalmatians,2000,85000000,183600000,1.160000,22-Nov-00
2,b48,w147,,12 years a slave,2013,17100000,187700000,9.976608,18-Oct-13
3,b50,w302,o7878,127 hours,yes,18000000,60700000,2.372222,5-Nov-10
4,b55,w160,,13 hours: the secret soldiers of benghazi,2016,50000000,70000000,0.400000,15-Jan-16


In [32]:
# Converting release date into datetime data type
import datetime as dt

for r in range(0,movie_df_3.shape[0]):
    if(len(movie_df_3.loc[r,'release_date'])>9):
        movie_df_3.loc[r,'release_date'] = ""
    if(len(movie_df_3.loc[r,'release_date'])>=8):   
        movie_df_3.loc[r,'release_date'] = pd.to_datetime(movie_df_3.loc[r,'release_date'], format='%d-%b-%y')
    else:
        movie_df_3.loc[r,'release_date'] = ""

movie_df_3.head()

,key_bom,key_wiki,key_omdb,title,year,budget,gross,revenue,release_date
0,b27,w123,,"""""""10,000 bc """"""",2008,105000000,269800000,1.569524,2008-03-07 00:00:00
1,b38,w299,o10558,102 dalmatians,2000,85000000,183600000,1.160000,2000-11-22 00:00:00
2,b48,w147,,12 years a slave,2013,17100000,187700000,9.976608,2013-10-18 00:00:00
3,b50,w302,o7878,127 hours,yes,18000000,60700000,2.372222,2010-11-05 00:00:00
4,b55,w160,,13 hours: the secret soldiers of benghazi,2016,50000000,70000000,0.400000,2016-01-15 00:00:00


In [33]:
# Get the month as a feature
from datetime import datetime 
for r in range(0,movie_df_3.shape[0]):
    if(movie_df_3.loc[r,'release_date']!=""):
        movie_df_3.loc[r,'release_month'] = datetime.strftime(movie_df_3.loc[r,'release_date'],"%m")
    else:
        movie_df_3.loc[r,'release_month'] = ""
        
# Get the quarter
for r in range(0,movie_df_3.shape[0]):
    if(movie_df_3.loc[r,'release_date']!=""):
        movie_df_3.loc[r,'release_quarter'] = movie_df_3.loc[r,'release_date'].quarter
    else:
        movie_df_3.loc[r,'release_quarter'] = ""
        
# Get the day of the year
for r in range(0,movie_df_3.shape[0]):
    if(movie_df_3.loc[r,'release_date']!=""):
        movie_df_3.loc[r,'release_day_of_the_year'] = datetime.strftime(movie_df_3.loc[r,'release_date'],"%j").lstrip("0")
    else:
        movie_df_3.loc[r,'release_day_of_the_year'] = ""

# get the week of the year
for r in range(0,movie_df_3.shape[0]):
    if(movie_df_3.loc[r,'release_date']!=""):
        movie_df_3.loc[r,'release_week_of_the_year'] = datetime.strftime(movie_df_3.loc[r,'release_date'],"%U").lstrip("0")
    else:
        movie_df_3.loc[r,'release_week_of_the_year'] = ""


print(movie_df_3.shape)
movie_df_3.head()

(6754, 13)


,key_bom,key_wiki,key_omdb,title,year,budget,gross,revenue,release_date,release_month,release_quarter,release_day_of_the_year,release_week_of_the_year
0,b27,w123,,"""""""10,000 bc """"""",2008,105000000,269800000,1.569524,2008-03-07 00:00:00,03,1,67,9
1,b38,w299,o10558,102 dalmatians,2000,85000000,183600000,1.160000,2000-11-22 00:00:00,11,4,327,47
2,b48,w147,,12 years a slave,2013,17100000,187700000,9.976608,2013-10-18 00:00:00,10,4,291,41
3,b50,w302,o7878,127 hours,yes,18000000,60700000,2.372222,2010-11-05 00:00:00,11,4,309,44
4,b55,w160,,13 hours: the secret soldiers of benghazi,2016,50000000,70000000,0.400000,2016-01-15 00:00:00,01,1,15,2


In [34]:
movie_df = movie_df_3
movie_df.to_csv("movies_final.csv",index=None)

##### Feature: holiday season

In [36]:
from pandas.tseries.holiday import USFederalHolidayCalendar
import datetime
from datetime import timedelta

cal = USFederalHolidayCalendar()
holidays = cal.holidays(start='1900-01-01', end='2017-12-31').to_pydatetime()

holidays_new = list(holidays)
print(len(holidays_new))

for h in holidays:
    holidays_new.append(h + datetime.timedelta(days=7))
    holidays_new.append(h + datetime.timedelta(days=6))
    holidays_new.append(h + datetime.timedelta(days=5))
    holidays_new.append(h + datetime.timedelta(days=4))
    holidays_new.append(h + datetime.timedelta(days=3))
    holidays_new.append(h + datetime.timedelta(days=2))
    holidays_new.append(h + datetime.timedelta(days=1))
    
    holidays_new.append(h - datetime.timedelta(days=7))
    holidays_new.append(h - datetime.timedelta(days=6))
    holidays_new.append(h - datetime.timedelta(days=5))
    holidays_new.append(h - datetime.timedelta(days=4))
    holidays_new.append(h - datetime.timedelta(days=3))
    holidays_new.append(h - datetime.timedelta(days=2))
    holidays_new.append(h - datetime.timedelta(days=1))
    
print(len(holidays_new))


1094
16410


In [37]:
t = movie_df
t = t.fillna("")


for r in range(0,t.shape[0]):
    if(t.loc[r,'release_date']!=""):
        if pd.to_datetime(t.loc[r,'release_date']) in holidays_new:
            t.loc[r,'holiday_season'] = 1
        else:
            t.loc[r,'holiday_season'] = 0
    else:
        t.loc[r,'holiday_season'] = ""
        
print(t.shape)
t.head()

(6754, 14)


,key_bom,key_wiki,key_omdb,title,year,budget,gross,revenue,release_date,release_month,release_quarter,release_day_of_the_year,release_week_of_the_year,holiday_season
0,b27,w123,,"""""""10,000 bc """"""",2008,105000000,269800000,1.569524,2008-03-07 00:00:00,3,1,67,9,0
1,b38,w299,o10558,102 dalmatians,2000,85000000,183600000,1.160000,2000-11-22 00:00:00,11,4,327,47,1
2,b48,w147,,12 years a slave,2013,17100000,187700000,9.976608,2013-10-18 00:00:00,10,4,291,41,1
3,b50,w302,o7878,127 hours,yes,18000000,60700000,2.372222,2010-11-05 00:00:00,11,4,309,44,1
4,b55,w160,,13 hours: the secret soldiers of benghazi,2016,50000000,70000000,0.400000,2016-01-15 00:00:00,1,1,15,2,1


In [38]:
print("No:of movies thay for teleased during holiday season")
t[t['holiday_season']==1].shape

No:of movies thay for teleased during holiday season


(1492, 14)

In [39]:
movie_df = t
movie_df.to_csv("movies_final.csv",index=None)

###### Statistical Model: simple linear regression

In [45]:
# Importing the library for regression
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Fitting a linear regression model 
results = smf.ols(formula = 'revenue ~ holiday_season + release_week_of_the_year + release_quarter + release_month+ release_day_of_the_year', 
                  data = movie_df).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                revenue   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.9359
Date:                Wed, 03 May 2017   Prob (F-statistic):              0.456
Time:                        06:56:23   Log-Likelihood:            -1.2479e+05
No. Observations:                4969   AIC:                         2.496e+05
Df Residuals:                    4963   BIC:                         2.496e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------------
Intercept                -1.217e+09   9.39e+08     -1.296      0.195     -3.06e+09  6.24e+08
holiday_season            8.566e+08   6.22e+08      1.377      0.168     -3.63e+08  2.08e+09
release_week_of_the_year -2.228e+08    8.8e+08     -0.253      0.800     -1.95e+09   1.5e+09
release_quarter           1.742e+08   1.03e+09      0.169      0.866     -1.85e+09   2.2e+09
release_month             1.207e+09   1.04e+09      1.156      0.248      -8.4e+08  3.25e+09
release_day_of_the_year  -7.202e+06    1.3e+08     -0.056      0.956     -2.61e+08  2.47e+08
==============================================================================
Omnibus:                    17684.375   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):       5086667887.231
Skew:                          70.370   Prob(JB):                         0.00
Kurtosis:                    4957.646   Cond. No.                         983.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [46]:
movie_df.corr()

,budget,gross,revenue,release_month,release_quarter,release_day_of_the_year,release_week_of_the_year,holiday_season
budget,1.000000,-0.000221,-0.000221,-0.014381,-0.013631,-0.014865,-0.014090,0.031744
gross,-0.000221,1.000000,1.000000,0.016612,0.017085,0.015230,0.015140,0.021680
revenue,-0.000221,1.000000,1.000000,0.016612,0.017085,0.015230,0.015140,0.021679
release_month,-0.014381,0.016612,0.016612,1.000000,0.970203,0.996520,0.996290,0.200820
release_quarter,-0.013631,0.017085,0.017085,0.970203,1.000000,0.966407,0.966064,0.210528
release_day_of_the_year,-0.014865,0.015230,0.015230,0.996520,0.966407,1.000000,0.999771,0.205294
release_week_of_the_year,-0.014090,0.015140,0.015140,0.996290,0.966064,0.999771,1.000000,0.210256
holiday_season,0.031744,0.021680,0.021679,0.200820,0.210528,0.205294,0.210256,1.000000


### Saurabh